---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-data-analysis/resources/0dhYG) course resource._

---

In [1]:
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind

# Assignment 4 - Hypothesis Testing
This assignment requires more individual learning than previous assignments - you are encouraged to check out the [pandas documentation](http://pandas.pydata.org/pandas-docs/stable/) to find functions or methods you might not have used yet, or ask questions on [Stack Overflow](http://stackoverflow.com/) and tag them as pandas and python related. And of course, the discussion forums are open for interaction with your peers and the course staff.

Definitions:
* A _quarter_ is a specific three month period, Q1 is January through March, Q2 is April through June, Q3 is July through September, Q4 is October through December.
* A _recession_ is defined as starting with two consecutive quarters of GDP decline, and ending with two consecutive quarters of GDP growth.
* A _recession bottom_ is the quarter within a recession which had the lowest GDP.
* A _university town_ is a city which has a high percentage of university students compared to the total population of the city.

**Hypothesis**: University towns have their mean housing prices less effected by recessions. Run a t-test to compare the ratio of the mean price of houses in university towns the quarter before the recession starts compared to the recession bottom. (`price_ratio=quarter_before_recession/recession_bottom`)

The following data files are available for this assignment:
* From the [Zillow research data site](http://www.zillow.com/research/data/) there is housing data for the United States. In particular the datafile for [all homes at a city level](http://files.zillowstatic.com/research/public/City/City_Zhvi_AllHomes.csv), ```City_Zhvi_AllHomes.csv```, has median home sale prices at a fine grained level.
* From the Wikipedia page on college towns is a list of [university towns in the United States](https://en.wikipedia.org/wiki/List_of_college_towns#College_towns_in_the_United_States) which has been copy and pasted into the file ```university_towns.txt```.
* From Bureau of Economic Analysis, US Department of Commerce, the [GDP over time](http://www.bea.gov/national/index.htm#gdp) of the United States in current dollars (use the chained value in 2009 dollars), in quarterly intervals, in the file ```gdplev.xls```. For this assignment, only look at GDP data from the first quarter of 2000 onward.

Each function in this assignment below is worth 10%, with the exception of ```run_ttest()```, which is worth 50%.

In [2]:
# Use this dictionary to map state names to two letter acronyms
states = {'OH': 'Ohio', 'KY': 'Kentucky', 'AS': 'American Samoa', 'NV': 'Nevada', 'WY': 'Wyoming', 'NA': 'National', 'AL': 'Alabama', 'MD': 'Maryland', 'AK': 'Alaska', 'UT': 'Utah', 'OR': 'Oregon', 'MT': 'Montana', 'IL': 'Illinois', 'TN': 'Tennessee', 'DC': 'District of Columbia', 'VT': 'Vermont', 'ID': 'Idaho', 'AR': 'Arkansas', 'ME': 'Maine', 'WA': 'Washington', 'HI': 'Hawaii', 'WI': 'Wisconsin', 'MI': 'Michigan', 'IN': 'Indiana', 'NJ': 'New Jersey', 'AZ': 'Arizona', 'GU': 'Guam', 'MS': 'Mississippi', 'PR': 'Puerto Rico', 'NC': 'North Carolina', 'TX': 'Texas', 'SD': 'South Dakota', 'MP': 'Northern Mariana Islands', 'IA': 'Iowa', 'MO': 'Missouri', 'CT': 'Connecticut', 'WV': 'West Virginia', 'SC': 'South Carolina', 'LA': 'Louisiana', 'KS': 'Kansas', 'NY': 'New York', 'NE': 'Nebraska', 'OK': 'Oklahoma', 'FL': 'Florida', 'CA': 'California', 'CO': 'Colorado', 'PA': 'Pennsylvania', 'DE': 'Delaware', 'NM': 'New Mexico', 'RI': 'Rhode Island', 'MN': 'Minnesota', 'VI': 'Virgin Islands', 'NH': 'New Hampshire', 'MA': 'Massachusetts', 'GA': 'Georgia', 'ND': 'North Dakota', 'VA': 'Virginia'}

In [3]:
def get_list_of_university_towns():
    '''Returns a DataFrame of towns and the states they are in from the 
    university_towns.txt list. The format of the DataFrame should be:
    DataFrame( [ ["Michigan", "Ann Arbor"], ["Michigan", "Yipsilanti"] ], 
    columns=["State", "RegionName"]  )
    
    The following cleaning needs to be done:

    1. For "State", removing characters from "[" to the end.
    2. For "RegionName", when applicable, removing every character from " (" to the end.
    3. Depending on how you read the data, you may need to remove newline character '\n'. '''

    df = pd.read_table('university_towns.txt', sep='\n', header=None, names=['Raw_Data'])
    
    sub_string1 = '[ed'
    a1 = df['Raw_Data'].str.find(sub_string1)
    
    for i in range(0,567,1) :
        if a1[i] > 0 :
            df.at[i, 'State'] = df.iloc[i]['Raw_Data'][0:a1[i]]
            
    df['State'] = df['State'].fillna(method='ffill')
    
    sub_string2 = '('
    a2 = df['Raw_Data'].str.find(sub_string2)
    
    for i in range(0,567,1) :
        if a2[i] > 0 :
            df.at[i, 'RegionName'] = df.iloc[i]['Raw_Data'][0:a2[i] - 1]
            
    a4 = df.isnull()
    sub_string3 = ':'
    a3 = df['Raw_Data'].str.find(sub_string3)
    
    for i in range(0,567,1) :
        if (a4.iloc[i]['RegionName'] == True) and (a3[i] > 0) :
            df.at[i, 'RegionName'] = df.iloc[i]['Raw_Data'][0:a3[i] + 1]
    
    a4 = df.isnull()
    
    for i in range(0,567,1) :
        if (a4.iloc[i]['RegionName'] == True) and (a1[i] < 0) :
            df.iloc[i]['RegionName'] = df.iloc[i]['Raw_Data']
    
    df = df[['State', 'RegionName']]
    df = df.dropna()
    df = df.reset_index(drop=True)
    answer1 = df

    return answer1

In [4]:
def get_recession_start():
    '''Returns the year and quarter of the recession start time as a 
    string value in a format such as 2005q3'''
    
    df1 = pd.read_excel('gdplev.xls', usecols=(4,6), skiprows=213, header=6)
    df1.columns = ['Quarter', 'GDP']
    
    qtr = []
    
    for i in range(1,65,1) :
        if (df1.iloc[i]['GDP'] < df1.iloc[i - 1]['GDP']) and (df1.iloc[i - 1]['GDP'] < df1.iloc[i - 2]['GDP']) :
            qtr.append(df1.iloc[i - 1]['Quarter'])

    qtr[0]
    answer2 = qtr[0]
    
    return answer2

In [5]:
def get_recession_end():
    '''Returns the year and quarter of the recession end time as a 
    string value in a format such as 2005q3'''
    
    df1 = pd.read_excel('gdplev.xls', usecols=(4,6), skiprows=213, header=6)
    df1.columns = ['Quarter', 'GDP']
    
    qtr1 = []
    
    a = (df1['Quarter'] == get_recession_start())
    
    for i in range(0,66,1) :
        if a[i] == True :
            start_rec = df1.index[i]
            
    for i in range(start_rec,66,1) :
        if (df1.iloc[i]['GDP'] > df1.iloc[i - 1]['GDP']) and (df1.iloc[i - 1]['GDP'] > df1.iloc[i - 2]['GDP']) :
            qtr1.append(df1.iloc[i]['Quarter'])

    answer3 = qtr1[0]
    return answer3

In [6]:
def get_recession_bottom():
    '''Returns the year and quarter of the recession bottom time as a 
    string value in a format such as 2005q3'''
    
    df1 = pd.read_excel('gdplev.xls', usecols=(4,6), skiprows=213, header=6)
    df1.columns = ['Quarter', 'GDP']
    
    df1 = df1[(df1['Quarter'] > '2008q2') & (df1['Quarter'] < '2010q1')]
    bttm_rec = min(df1['GDP'])
    
    for i in range(0,6,1) :
        if df1.iloc[i]['GDP'] == bttm_rec :
            answer4 = df1.iloc[i]['Quarter']

    return answer4

In [7]:
def convert_housing_data_to_quarters():
    '''Converts the housing data to quarters and returns it as mean 
    values in a dataframe. This dataframe should be a dataframe with
    columns for 2000q1 through 2016q3, and should have a multi-index
    in the shape of ["State","RegionName"].
    
    Note: Quarters are defined in the assignment description, they are
    not arbitrary three month periods.
    
    The resulting dataframe should have 67 columns, and 10,730 rows.
    '''
    ColumnCount = [1, 2]

    for i in range (51,251,1) :
        ColumnCount.append(i)

    df2 = pd.read_csv('City_Zhvi_AllHomes.csv', header=0, usecols=ColumnCount)

    df3 = df2[['State', 'RegionName']]
    df3 = df3.replace({'State':states})
    df3 = df3[['State', 'RegionName']]

    for year in range(2000,2016):
        df3[str(year)+'q1'] = df2[[str(year)+'-01',str(year)+'-02',str(year)+'-03']].mean(axis=1)
        df3[str(year)+'q2'] = df2[[str(year)+'-04',str(year)+'-05',str(year)+'-06']].mean(axis=1)
        df3[str(year)+'q3'] = df2[[str(year)+'-07',str(year)+'-08',str(year)+'-09']].mean(axis=1)
        df3[str(year)+'q4'] = df2[[str(year)+'-10',str(year)+'-11',str(year)+'-12']].mean(axis=1)

    year = 2016
    df3[str(year)+'q1'] = df2[[str(year)+'-01',str(year)+'-02',str(year)+'-03']].mean(axis=1)
    df3[str(year)+'q2'] = df2[[str(year)+'-04',str(year)+'-05',str(year)+'-06']].mean(axis=1)
    df3[str(year)+'q3'] = df2[[str(year)+'-07',str(year)+'-08']].mean(axis=1)

    df3 = df3.set_index(['State','RegionName'])
    answer5 = df3
    return answer5

In [8]:
def run_ttest():
    '''First creates new data showing the decline or growth of housing prices
    between the recession start and the recession bottom. Then runs a ttest
    comparing the university town values to the non-university towns values, 
    return whether the alternative hypothesis (that the two groups are the same)
    is true or not as well as the p-value of the confidence. 
    
    Return the tuple (different, p, better) where different=True if the t-test is
    True at a p<0.01 (we reject the null hypothesis), or different=False if 
    otherwise (we cannot reject the null hypothesis). The variable p should
    be equal to the exact p value returned from scipy.stats.ttest_ind(). The
    value for better should be either "university town" or "non-university town"
    depending on which has a lower mean price ratio (which is equivilent to a
    reduced market loss).'''
    
    UniversityTowns = get_list_of_university_towns()
    RecessionStart = get_recession_start()
    RecessionBottom = get_recession_bottom()
    HousingData = convert_housing_data_to_quarters()

    Column_1 = HousingData.columns[HousingData.columns.get_loc(RecessionStart) - 1]

    df4 = HousingData[[Column_1, RecessionBottom]]
    df4['Calculation'] = HousingData[RecessionBottom] - HousingData[Column_1]
    df4 = df4.reset_index()

    UniversityTownsRecession = pd.merge(df4, UniversityTowns, on=['State', 'RegionName'], how='inner')
    NonUniversityTownsRecession = df4.merge(UniversityTowns,indicator = True, how='left').loc[lambda x : x['_merge']!='both']
    NonUniversityTownsRecession = NonUniversityTownsRecession.drop('_merge', axis=1)

    stat = ttest_ind(UniversityTownsRecession['Calculation'].dropna(), NonUniversityTownsRecession['Calculation'].dropna())

    if UniversityTownsRecession.mean()[0] < NonUniversityTownsRecession.mean()[0] :
        answer_63 = 'university town'

    else :
        answer_63 = 'non-university town'

    if stat[1] < 0.01 :
        answer6 = (True, stat[1], answer_63)

    else :
        answer6 = (False, stat[1], answer_63)

    return answer6